In [17]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
import pickle

In [18]:
data = pd.read_csv('../dataset/Churn_Modelling.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [19]:
data = data.drop(['RowNumber', 'CustomerId','Surname'], axis=1)
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [20]:
label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])

onehot_encoder_geo = OneHotEncoder(handle_unknown="ignore")
geo_encoded = onehot_encoder_geo.fit_transform(data[['Geography']]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=onehot_encoder_geo.get_feature_names_out(['Geography']))

data = pd.concat([data.drop("Geography", axis=1), geo_encoded_df], axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [21]:
X = data.drop("EstimatedSalary", axis=1)
Y = data["EstimatedSalary"]

In [22]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [23]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [24]:
with open('../models/salaryPredictor/label_encoder_gender.pkl', 'wb') as file:
    pickle.dump(label_encoder_gender, file)

with open('../models/salaryPredictor/onehot_encoder_geo.pkl', 'wb') as file:
    pickle.dump(onehot_encoder_geo, file)

with open('../models/salaryPredictor/scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

In [25]:
## ANN with regression
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [26]:
model = Sequential(
    [
        Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
        Dense(32, activation='relu'),
        Dense(1),
    ]
)

model.compile(optimizer='adam', loss='mean_absolute_error', metrics="mae")
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 64)                832       
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 dense_5 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2,945
Trainable params: 2,945
Non-trainable params: 0
_________________________________________________________________


In [27]:
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [28]:
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [29]:
history = model.fit(
    X_train_scaled, Y_train,
    epochs=100,
    validation_data=(X_test_scaled, Y_test),
    callbacks=[early_stopping_callback, tensorboard_callback]
)

Epoch 1/100
250/250 [==============================] - 1s 2ms/step - loss: 100396.5469 - mae: 100396.5469 - val_loss: 98586.1406 - val_mae: 98586.1406
Epoch 2/100
250/250 [==============================] - 0s 1ms/step - loss: 99862.8672 - mae: 99862.8672 - val_loss: 97486.7266 - val_mae: 97486.7266
Epoch 3/100
250/250 [==============================] - 0s 1ms/step - loss: 97911.0234 - mae: 97911.0234 - val_loss: 94589.5859 - val_mae: 94589.5859
Epoch 4/100
250/250 [==============================] - 0s 1ms/step - loss: 93971.4766 - mae: 93971.4766 - val_loss: 89617.4062 - val_mae: 89617.4062
Epoch 5/100
250/250 [==============================] - 0s 1ms/step - loss: 87998.7578 - mae: 87998.7578 - val_loss: 82842.7891 - val_mae: 82842.7891
Epoch 6/100
250/250 [==============================] - 0s 1ms/step - loss: 80454.7500 - mae: 80454.7500 - val_loss: 75054.7969 - val_mae: 75054.7969
Epoch 7/100
250/250 [==============================] - 0s 1ms/step - loss: 72370.7812 - mae: 72370.7812 

In [30]:
%load_ext tensorboard
%tensorboard --logdir logs/fit/

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 8124), started 0:00:43 ago. (Use '!kill 8124' to kill it.)

In [31]:
## Evaluvate model on the test data

test_loss, test_mae = model.evaluate(X_test_scaled, Y_test, verbose=2)
print(f"Test mae: {test_mae:.2f}")

63/63 - 0s - loss: 50219.2422 - mae: 50219.2422 - 82ms/epoch - 1ms/step
Test mae: 50219.24


In [32]:
model.save('../models/salaryPredictor/regression_model.h5')